In [1]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline

In [2]:
from glob import glob

In [3]:
images = glob('../datasets/male_vs_female_classification/images/*.jpg')

In [4]:
images

['../datasets/male_vs_female_classification/images/15193.jpg',
 '../datasets/male_vs_female_classification/images/14692.jpg',
 '../datasets/male_vs_female_classification/images/9898.jpg',
 '../datasets/male_vs_female_classification/images/1770.jpg',
 '../datasets/male_vs_female_classification/images/1314.jpg',
 '../datasets/male_vs_female_classification/images/8112.jpg',
 '../datasets/male_vs_female_classification/images/8143.jpg',
 '../datasets/male_vs_female_classification/images/8097.jpg',
 '../datasets/male_vs_female_classification/images/10873.jpg',
 '../datasets/male_vs_female_classification/images/14525.jpg',
 '../datasets/male_vs_female_classification/images/8177.jpg',
 '../datasets/male_vs_female_classification/images/21013.jpg',
 '../datasets/male_vs_female_classification/images/26581.jpg',
 '../datasets/male_vs_female_classification/images/23558.jpg',
 '../datasets/male_vs_female_classification/images/341.jpg',
 '../datasets/male_vs_female_classification/images/24313.jpg',
 

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('../datasets/male_vs_female_classification/male_vs_female_classification.csv')

In [7]:
data.head()

,image_name,class
0,1267.jpg,1
1,1268.jpg,1
2,1269.jpg,1
3,1270.jpg,1
4,1271.jpg,1


In [8]:
data['class'].value_counts()

1    8853
0    8570
Name: class, dtype: int64

In [9]:
# create an empty python list
X = []
for img_name in data.image_name:
    img = plt.imread('../datasets/male_vs_female_classification/images/' + img_name)
    X.append(img)
    
# convert this python list to a single numpy array
X = np.array(X)

In [10]:
X.shape

(17423, 224, 224, 3)

In [11]:
y = data['class'].values
y.shape

(17423,)

In [12]:
from keras.utils import to_categorical

y = to_categorical(y)
y.shape

Using TensorFlow backend.


(17423, 2)

In [13]:
from keras.applications.vgg16 import preprocess_input

In [14]:
X = preprocess_input(X, mode='tf')

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

In [19]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [20]:
base_model_pred = base_model.predict(X_train)
base_model_pred_valid = base_model.predict(X_valid)

In [21]:
base_model_pred.shape

(12196, 7, 7, 512)

In [23]:
base_model_pred = base_model_pred.reshape(12196, 7*7*512)

In [24]:
base_model_pred_valid.shape

(5227, 7, 7, 512)

In [25]:
base_model_pred_valid = base_model_pred_valid.reshape(5227, 7*7*512)

In [26]:
base_model_pred.max(), base_model_pred_valid.max()

(11.018945, 11.118778)

In [27]:
max_val = base_model_pred.max()

base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val

In [28]:
base_model_pred.min(), base_model_pred.max(), base_model_pred.mean()

(0.0, 1.0, 0.011371137)

In [29]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [30]:
model = Sequential()
model.add(InputLayer((7*7*512, )))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [31]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(base_model_pred, Y_train, epochs=15, validation_data=(base_model_pred_valid, Y_valid))

Train on 12196 samples, validate on 5227 samples
Epoch 1/15
12196/12196 [==============================] - 7s 538us/step - loss: 0.4286 - acc: 0.7903 - val_loss: 0.3466 - val_acc: 0.8385
Epoch 2/15
12196/12196 [==============================] - 6s 495us/step - loss: 0.2933 - acc: 0.8691 - val_loss: 0.3163 - val_acc: 0.8596
Epoch 3/15
12196/12196 [==============================] - 6s 497us/step - loss: 0.2373 - acc: 0.8968 - val_loss: 0.3258 - val_acc: 0.8533
Epoch 4/15
12196/12196 [==============================] - 6s 497us/step - loss: 0.1901 - acc: 0.9173 - val_loss: 0.3236 - val_acc: 0.8670
Epoch 5/15
12196/12196 [==============================] - 6s 501us/step - loss: 0.1531 - acc: 0.9361 - val_loss: 0.8237 - val_acc: 0.7368
Epoch 6/15
12196/12196 [==============================] - 6s 497us/step - loss: 0.1471 - acc: 0.9393 - val_loss: 0.3327 - val_acc: 0.8755
Epoch 7/15
12196/12196 [==============================] - 6s 497us/step - loss: 0.1253 - acc: 0.9490 - val_loss: 0.4150 - v

In [21]:
for layer in base_model.layers:
    layer.Trainable = False

In [22]:
model = Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,717,249
Trainable params: 14,716,225
Non-trainable params: 1,024
_________________________________________________________________


In [ ]:
model.fit(X_train, Y_train, epochs=3, validation_data=(X_valid, Y_valid))

Train on 20356 samples, validate on 8724 samples
Epoch 1/3
 6528/20356 [========>.....................] - ETA: 1:58 - loss: 0.5607 - acc: 0.7157